<a href="https://colab.research.google.com/github/As3n3n/As95asteroid/blob/main/YouTube%E8%A1%A8%E7%94%9F%E6%88%90%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# ==== 事前インストール ====
!pip install yt-dlp tabulate --quiet

import os
import json
import time
import random
import shutil
from datetime import datetime
from IPython.display import display, HTML
from google.colab import files
import yt_dlp

# ==== 設定 ====
RETRY_LIMIT = 3
SLEEP_BETWEEN = (2, 4)
CACHE_FILE = "video_cache.json"
BACKUP_DIR = "backups"
MAX_BACKUPS = 3
OWN_CHANNEL_NAME = "ペイリアン_peilien"

# ==== ユーティリティ ====
def sleep_random():
    time.sleep(random.uniform(*SLEEP_BETWEEN))

def retry(func, *args, **kwargs):
    for attempt in range(RETRY_LIMIT):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"リトライ {attempt+1}/{RETRY_LIMIT} エラー: {e}")
            sleep_random()
    raise

# ==== キャッシュ管理 ====
def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r', encoding='utf-8') as f:
            try:
                return json.load(f)
            except Exception:
                print("キャッシュ読み込み失敗")
    return {}

def save_cache(cache):
    with open(CACHE_FILE, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

def clear_cache():
    if os.path.exists(CACHE_FILE):
        os.remove(CACHE_FILE)
        print("キャッシュを削除しました。")

def backup_cache():
    if not os.path.exists(CACHE_FILE):
        return
    os.makedirs(BACKUP_DIR, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    path = os.path.join(BACKUP_DIR, f'cache_{timestamp}.json')
    shutil.copy(CACHE_FILE, path)
    backups = sorted(os.listdir(BACKUP_DIR))
    while len(backups) > MAX_BACKUPS:
        os.remove(os.path.join(BACKUP_DIR, backups.pop(0)))

# ==== 軽量キャッシュエクスポート/インポート ====
def export_minimal_cache():
    full = load_cache()
    minimal = {}
    for vid, info in full.items():
        minimal[vid] = {
            'title': info.get('title'),
            'upload_date': info.get('upload_date'),
            'duration': info.get('duration'),
            'uploader': info.get('uploader'),
            'thumbnail': info.get('thumbnail'),
            'webpage_url': info.get('webpage_url')
        }
    fname = 'minimal_cache.json'
    with open(fname, 'w', encoding='utf-8') as f:
        json.dump(minimal, f, ensure_ascii=False, indent=2)
    files.download(fname)
    print(f"軽量キャッシュをエクスポートしました: {fname}")

def import_minimal_cache():
    uploaded = files.upload()
    for fn in uploaded:
        with open(fn, 'r', encoding='utf-8') as f:
            minimal = json.load(f)
        full = load_cache()
        full.update(minimal)
        save_cache(full)
        print(f"軽量キャッシュをインポートし統合しました: {fn}")

# ==== テーブル表示 ====
def display_copyable_table(text):
    html = f"""
    <textarea id=\"copyArea\" rows=20 style=\"width:100%;font-family:monospace;\">{text}</textarea><br>
    <button onclick=\"navigator.clipboard.writeText(document.getElementById('copyArea').value)\">コピー</button>
    """
    display(HTML(html))

# ==== データ取得 ====
def fetch_video_info(url):
    ydl_opts = {'quiet': True, 'extract_flat': True, 'skip_download': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        return retry(ydl.extract_info, url, download=False)

# ==== メイン処理 ====
def main():
    # インポート選択
    print("キャッシュをインポートしますか？ full=フル/minimal=軽量/Enter=スキップ")
    choice = input().strip().lower()
    if choice == 'full':
        uploaded = files.upload()
        for fn in uploaded:
            shutil.copy(fn, CACHE_FILE)
        print("フルキャッシュをインポートしました。")
    elif choice == 'minimal':
        import_minimal_cache()

    # URL入力
    url = input("YouTube URL（再生リスト or 動画）> ").strip()

    # キャッシュ削除選択
    if input("キャッシュを削除しますか？ yes で削除／Enterでスキップ > ").strip().lower() == 'yes':
        clear_cache()

    cache = load_cache()
    info = fetch_video_info(url)
    entries = info.get('entries', [info])

    # テーブル組み立て
    table = [["投稿日", "再生時間", "タイトル", "備考", "チャンネル|c"]]
    total = len(entries)
    for i, entry in enumerate(entries, 1):
        vid = entry.get('id')
        if not vid:
            continue
        print(f"[{i}/{total}] {vid}")
        start = time.time()
        if vid in cache:
            data = cache[vid]
            print("  → キャッシュ済み")
        else:
            sleep_random()
            data = retry(yt_dlp.YoutubeDL({'quiet':True}).extract_info, f"https://www.youtube.com/watch?v={vid}", download=False)
            cache[vid] = data
            save_cache(cache)
            backup_cache()
        elapsed = time.time() - start
        print(f"  → {elapsed:.2f}秒")

        # データ整形
        date = data.get('upload_date','')
        date = f"{date[:4]}/{date[4:6]}/{date[6:]}" if len(date)==8 else ''
        dur = data.get('duration',0)
        dur_str = f"{dur//3600}:{(dur%3600)//60:02}:{dur%60:02}" if dur>=3600 else f"{dur//60}:{dur%60:02}"
        title = data.get('title','')
        thumb = data.get('thumbnail','')
        link = data.get('webpage_url', '')
        chan = data.get('uploader','')
        if chan == OWN_CHANNEL_NAME:
            chan = "自チャンネル"
        wiki = f'[[{title}~~&ref({thumb},60%)>>{link}]]'
        table.append([date, dur_str, wiki, '', chan])

    # 出力
    lines = []
    for idx, row in enumerate(table):
        line = '|' + '|'.join(row)
        if idx > 0:
            line += '|'  # データ行末尾の |
        lines.append(line)
    text = '\n'.join(lines)
    display_copyable_table(text)

    # エクスポート選択
    print("キャッシュをエクスポートしますか？ full=フル/minimal=軽量/Enter=終了")
    choice = input().strip().lower()
    if choice == 'full':
        files.download(CACHE_FILE)
    elif choice == 'minimal':
        export_minimal_cache()

# ==== 実行 ====
main()

キャッシュをインポートしますか？ full=フル/minimal=軽量/Enter=スキップ

YouTube URL（再生リスト or 動画）> https://youtube.com/playlist?list=PLuKKnCqP4WGd71ZcWnNmK9Mw3RDTkeGi1&si=hIySABE85IhaGHwq
キャッシュを削除しますか？ yes で削除／Enterでスキップ > 
[1/3] AFVFFYVmtuM
  → キャッシュ済み
  → 0.00秒
[2/3] PFlC4Qzp8Ko
  → キャッシュ済み
  → 0.00秒
[3/3] rWTo5b6mo1A
  → キャッシュ済み
  → 0.00秒


キャッシュをエクスポートしますか？ full=フル/minimal=軽量/Enter=終了
minimal


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

軽量キャッシュをエクスポートしました: minimal_cache.json
